In [1]:
!nvidia-smi

Sun Aug 10 07:43:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   70C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [34]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
rerank_df = pd.read_csv("C:\\Users\\SHREYA NARAYAN\\Downloads\\rerank_df_layer1 (4).csv")

In [36]:
item_popularity = pickle.load(open("item_popularity.pkl", "rb"))

In [37]:
cooccur = pickle.load(open("cooccur.pkl", "rb"))

In [38]:
orders_df= pd.read_csv("C:\\Users\\SHREYA NARAYAN\\Downloads\\orders_grouped_cleaned(1).csv")

In [39]:
test_df_with_candidates = pd.read_csv("C:\\Users\\SHREYA NARAYAN\\Downloads\\test_df_fb_candidates.csv")

In [40]:
def recall_at_k(df, k=3):
    correct = 0
    for _, row in df.groupby("ORDER_ID"):
        # Flatten actual items in case they're lists
        actual_items = []
        for val in row["actual_missing_item"]:
            if isinstance(val, list):
                actual_items.extend(val)
            else:
                actual_items.append(val)
        actual = set(actual_items)

        preds = row.sort_values("pred_score", ascending=False)["candidate_item"].head(k).tolist()
        if any(item in actual for item in preds):
            correct += 1
    return correct / df["ORDER_ID"].nunique()


In [41]:
baseline_recall = recall_at_k(
    rerank_df.merge(
        test_df_with_candidates[["ORDER_ID", "actual_missing_item"]],
        on="ORDER_ID",
        how="left"
    ),
    k=3
)

print(f"Baseline Recall@3: {baseline_recall:.4f}")


Baseline Recall@3: 0.0880


In [8]:
transactions_df = (
    orders_df.groupby("ORDER_ID")["item_name"]
    .apply(list)
    .reset_index(name="transactions")
) #We group based on order id and items

In [32]:
def recall_at_k(df, k=3):
    correct = 0
    for _, row in df.groupby("ORDER_ID"):
        actual = set(row["actual_missing_item"])
        preds = row.sort_values("pred_score", ascending=False)["candidate_item"].head(k).tolist()
        if any(item in actual for item in preds):
            correct += 1
    return correct / df["ORDER_ID"].nunique()

baseline_recall = recall_at_k(
    rerank_df.merge(
        test_df_with_candidates[["ORDER_ID", "actual_missing_item"]],
        on="ORDER_ID",
        how="left"
    ),
    k=3
)
print(f"Baseline Recall@3 before Layer 1.5: {baseline_recall:.4f}")

TypeError: unhashable type: 'list'

In [9]:
rerank_df = rerank_df.merge(transactions_df, on="ORDER_ID", how="left")

In [10]:
# 2) Merge contextual order info (no duplicates)
context_cols = [
    "ORDER_ID", "CUSTOMER_ID", "STORE_NUMBER",
    "DAY_NUMBER", "DAY_TYPE", "cart_items"
]
rerank_df = rerank_df.merge(test_df_with_candidates[context_cols], on="ORDER_ID", how="left")

In [11]:
def get_item_category(item):
    item = item.lower()
    if "fries" in item or "strips" in item or "sticks" in item:
        return "side"
    elif "drink" in item or "soda" in item or "beverage" in item or "20 oz" in item:
        return "drink"
    elif "combo" in item or "meal" in item or "feast" in item or "Pack" in item:
        return "combo"
    elif "wings" in item or "sub" in item or "crispy" in item:
        return "main"
    else:
        return "other"

def get_cart_category_vector(cart_items):
    return Counter([get_item_category(i) for i in cart_items])

In [12]:
# 1. Global Popularity Feature
rerank_df["global_popularity"] = rerank_df["candidate_item"].map(item_popularity).fillna(0)

In [13]:
# 2. Co-occurrence Score
def cooccur_score(cart_items, candidate):
    return sum(cooccur.get((i, candidate), 0) for i in cart_items)

rerank_df["cooccur_score"] = rerank_df.apply(
    lambda row: cooccur_score(row["cart_items"], row["candidate_item"]), axis=1
)

In [15]:
# 3. Store-Level Popularity
# Create the (day_type, item) → count dictionary
daytype_item_counts = (
    orders_df.groupby(["DAY_TYPE", "item_name"])
    .size()
    .reset_index(name="daytype_count")
)

# Convert to dict for O(1) lookups
daytype_dict = dict(zip(
    zip(daytype_item_counts["DAY_TYPE"], daytype_item_counts["item_name"]),
    daytype_item_counts["daytype_count"]
))

# Map directly
rerank_df["daytype_popularity"] = rerank_df.apply(
    lambda row: daytype_dict.get((row["DAY_TYPE"], row["candidate_item"]), 0),
    axis=1
)

In [16]:
# 4. Daytype (Weekend vs Weekday) Popularity
# Create the (store_number, item) → count dictionary
store_item_counts = (
    orders_df.groupby(["STORE_NUMBER", "item_name"])
    .size()
    .reset_index(name="store_count")
)

store_dict = dict(zip(
    zip(store_item_counts["STORE_NUMBER"], store_item_counts["item_name"]),
    store_item_counts["store_count"]
))

# Map directly
rerank_df["store_popularity"] = rerank_df.apply(
    lambda row: store_dict.get((row["STORE_NUMBER"], row["candidate_item"]), 0),
    axis=1
)

In [32]:
print(orders_df.columns)

Index(['ORDER_ID', 'CUSTOMER_ID', 'CUSTOMER_TYPE', 'STORE_NUMBER',
       'ORDER_CREATED_DATE', 'DAY_NAME', 'DAY_TYPE', 'ORDER_CHANNEL_NAME',
       'ORDER_OCCASION_NAME', 'num', 'ORDER_SUBCHANNEL_NAME', 'CITY', 'STATE',
       'POSTAL_CODE', 'item_name', 'total_order_amount', 'total_items'],
      dtype='object')


In [17]:
# 5. Last-minute Add-on Logic (Spicy → Drink boost)
spicy_keywords = ["spicy", "chili"]
drink_keywords = ["cola", "drink", "juice"]

def spicy_drink_boost(cart_items, candidate):
    cart_has_spicy = any(any(k in i.lower() for k in spicy_keywords) for i in cart_items)
    candidate_is_drink = any(k in candidate.lower() for k in drink_keywords)
    return 1 if cart_has_spicy and candidate_is_drink else 0

rerank_df["spicy_drink_boost"] = rerank_df.apply(
    lambda row: spicy_drink_boost(row["cart_items"], row["candidate_item"]), axis=1
)

In [18]:
# 6. Category Diversity Boost
def category_diversity_boost(cart_items, candidate):
    cart_cats = {get_item_category(i) for i in cart_items}
    candidate_cat = get_item_category(candidate)
    return 1 if candidate_cat not in cart_cats else 0

rerank_df["category_diversity_boost"] = rerank_df.apply(
    lambda row: category_diversity_boost(row["cart_items"], row["candidate_item"]), axis=1
)

In [21]:
# Ensure transactions column is always a list
rerank_df["transactions"] = rerank_df["transactions"].apply(
    lambda x: x if isinstance(x, list) else ([] if pd.isna(x) else [x])
)

# Category Mismatch Feature
rerank_df["cat_mismatch"] = rerank_df.apply(
    lambda row: int(
        get_item_category(row["candidate_item"]) not in 
        get_cart_category_vector(row["transactions"])
    ),
    axis=1
)

In [28]:
# Recall@3 Calculation
import ast
import pandas as pd

# Helper function to safely convert to list
def safe_to_list(x):
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    x_str = str(x).strip()
    if x_str.startswith("[") and x_str.endswith("]"):
        try:
            return ast.literal_eval(x_str)
        except:
            return []
    return [x_str]

K = 3  # Recall@K

# Clean up candidate list from rerank_df
rerank_df["candidate_item"] = rerank_df["candidate_item"].apply(safe_to_list)

# Load and clean actual missing items from test_df_fb
test_df_with_candidates["actual_missing_item"] = test_df_with_candidates["actual_missing_item"].apply(safe_to_list)

# Merge on ORDER_ID so we have both candidates & actuals in one place
eval_df = rerank_df.merge(
    test_df_with_candidates[["ORDER_ID", "actual_missing_item"]],
    on="ORDER_ID",
    how="inner"
)

# Calculate recall per order
recalls = []
for _, row in eval_df.iterrows():
    actual_items = set(row["actual_missing_item"])
    candidates = row["candidate_item"][:K]  # top K
    hit_count = len(set(candidates) & actual_items)
    recall = hit_count / len(actual_items) if actual_items else 0
    recalls.append(recall)

# Average recall@K
recall_at_k = sum(recalls) / len(recalls)
print(f"Recall@{K}: {recall_at_k:.4f}")


Recall@3: 0.0244


In [ ]:
rerank_df.to_pickle("/content/drive/MyDrive/menu_rec/rerank_df_layer1_5.pkl")
print(" Layer 1.5 features added. Saved to rerank_df_layer1_5.pkl")

In [29]:
import ast

# Load your candidate dataframe (stage 1 output)
# test_df_fb should have: ORDER_ID, candidate_item(s), actual_missing_item(s)
df = pd.read_csv("C:\\Users\\SHREYA NARAYAN\\Downloads\\test_df_fb_candidates.csv")

# Ensure lists are real Python lists
def safe_list(x):
    if isinstance(x, list):
        return x
    try:
        return ast.literal_eval(str(x))
    except:
        return [x]

df["candidate_item"] = df["candidate_item"].apply(safe_list)
df["actual_missing_item"] = df["actual_missing_item"].apply(safe_list)

# Parameters
K = 100  # candidate pool size you want to check

# Calculate recall@K for candidate generation
recalls = []
for _, row in df.iterrows():
    actual_items = row["actual_missing_item"]
    candidates = row["candidate_item"][:K]
    hit_count = len(set(candidates) & set(actual_items))
    recall = hit_count / len(actual_items) if actual_items else 0
    recalls.append(recall)

candidate_recall_at_k = sum(recalls) / len(recalls)
print(f"Candidate Recall@{K}: {candidate_recall_at_k:.4f}")


Candidate Recall@100: 0.0801
